In [1]:
#****************************************************************************************************#
# Mitigación de Errores en la Medida debido al ruido en Circuitos Cuánticos
#****************************************************************************************************#

#****************************************************************************************************#
# Para esta implementación se ha tenido en cuenta la descripción del circuito dada en el siguiente 
# tutorial del libro de Qiskit:
# https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/Coding_With_Qiskit/ep8_Noise_and_Error_Mitigation.ipynb
#****************************************************************************************************#

#****************************************************************************************************#
# Taller de Computación Cuántica
# Prof. Gustavo Patino (adolfo.patino@udea.edu.co)
# CORE 2022
# CIC - IPN 
# Ciudad de México - México
# Septiembre del 2022
#****************************************************************************************************#

In [2]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

In [3]:
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram

In [8]:
# First, see what devices we are allowed to use by loading our saved accounts
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='uni-antioquia-1', project='qc2021')
Remote_QProcessor = provider.get_backend('ibm_nairobi') # Maquina reservada temporalmente

In [9]:
#Remote_QProcessor = least_busy(provider.backends(filters=lambda qprocessor: qprocessor.configuration().n_qubits >= 2 
#                                       and not qprocessor.configuration().simulator 
#                                       and qprocessor.status().operational==True))
#print("El computador cuántico menos ocupado es el " + Remote_QProcessor.name())

In [10]:
pending=Remote_QProcessor.status().pending_jobs
print("La cantidad de ejecuciones (jobs) pendientes en este procesador real es: ", pending)

La cantidad de ejecuciones (jobs) pendientes en este procesador real es:  516


In [ ]:
# Descomente la siguiente linea si desea ver las imágenes más grandes.
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

In [ ]:
nqubits = 3
circuit = QuantumCircuit(nqubits, nqubits)

In [ ]:
circuit.h(0)
circuit.cx(0,1)
circuit.cx(1,2)
circuit.barrier()

In [ ]:
circuit.measure([0,1,2], [0,1,2])
circuit.draw(output='mpl')

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
sim_result = execute(circuit, backend=simulator, shots=1024).result()

In [ ]:
plot_histogram(sim_result.get_counts(circuit))

In [ ]:
job = execute(circuit, backend=Remote_QProcessor, shots=1024)
print('El id de esta ejecución (job) es: ', job.job_id()) # Este Id puede ser retomado para recuperar luego el job ejecutado.

In [ ]:
job_monitor(job)

In [ ]:
device_result = job.result()
plot_histogram(device_result.get_counts(circuit))

In [ ]:
# Definition of Calibration Circuits to mitigate the measurement error.

In [ ]:
from qiskit.utils.mitigation import (complete_meas_cal, tensored_meas_cal,
                                        CompleteMeasFitter, TensoredMeasFitter)

In [ ]:
calibration_circuits, quantum_states = complete_meas_cal(qr=circuit.qregs[0], 
                                               circlabel='measurement_calibration')

In [ ]:
quantum_states

In [ ]:
calibration_circuits[7].draw(output='mpl')

In [ ]:
len(calibration_circuits)

In [ ]:
cal_job = execute(calibration_circuits,
             backend=Remote_QProcessor,
             shots=8192,
             optimization_level=0)

print('El id de esta nueva ejecución (cal_job) es: ', cal_job.job_id()) # Este Id puede ser retomado para recuperar luego el job ejecutado.

In [ ]:
job_monitor(cal_job)

In [ ]:
calibration_results = cal_job.result()

In [ ]:
plot_histogram(calibration_results.get_counts(calibration_circuits[3]))

In [ ]:
My_fitter = CompleteMeasFitter(calibration_results, quantum_states)

In [ ]:
from qiskit.visualization import array_to_latex

print('La matriz de calibración para mitigar el error en la medida es:')
array_to_latex(My_fitter.cal_matrix)

In [ ]:
Measurement_Filter = My_fitter.filter

# Mitigating the measurement errors in our previous device run

In [ ]:
mitigated_result = Measurement_Filter.apply(device_result)

In [ ]:
device_counts = device_result.get_counts(circuit)
mitigated_counts = mitigated_result.get_counts(circuit)
plot_histogram([device_counts, mitigated_counts], 
               legend=['device, noisy', 'device, mitigated'])

# Running error mitigation on a second circuit

In [ ]:
circuit2 = QuantumCircuit(3,3)
circuit2.x(1)
circuit2.h(0)
circuit2.cx(0,1)
circuit2.cx(1,2)
circuit2.measure([0,1,2], [0,1,2])
circuit2.draw(output='mpl')

In [ ]:
# Simulating the circuit and plotting the corresponding histogram:
Simulation_counts_2 = execute(circuit2, backend=simulator, shots=1024).result().get_counts(circuit2)
plot_histogram(Simulation_counts_2)

In [ ]:
# Executing the circuit and plotting the corresponding histogram:
job2 = execute(circuit2, backend=Remote_QProcessor, shots=1024)
job_monitor(job2)

In [ ]:
device_counts_2=job2.result().get_counts(circuit2)
plot_histogram(device_counts_2)

In [ ]:
mitigated_counts_2 = Measurement_Filter.apply(device_counts_2)

In [ ]:
plot_histogram([device_counts_2, mitigated_counts_2], legend=['device, noisy','device, mitigated'])

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright